In [ ]:
import pandas as pd
import requests
import time
import re

# Leer el archivo de Spotify (ajusta el path si es necesario)
df_spotify = pd.read_csv("artist.csv")

artistas_raw = df_spotify['artist'].dropna().unique()
artistas = set()

# Función para separar artistas con múltiples delimitadores
def split_artists(texto):
    # Reemplazar "Featuring" por un delimitador temporal (e.g., ";") y limpiar
    texto = re.sub(r'\s*Featuring\s*', ',', texto, flags=re.IGNORECASE)
    # Separar por ";" y "&"
    partes = re.split(r'[;&]', texto)
    # Limpiar cada artista y devolver lista
    return [artista.strip() for artista in partes if artista.strip()]

# Procesar cada grupo de artistas
for grupo in artistas_raw:
    for artista in split_artists(grupo):
        artistas.add(artista)

print(f" Total de artistas únicos: {len(artistas)}")


🎵 Total de artistas únicos: 2691


In [ ]:
# Consultar MusicBrainz para cada artista
headers = {'User-Agent': 'ETLWorkshop/1.0 (tucorreo@dominio.com)'}
datos_artistas = []

for artist in list(artistas):
    query = artist.replace(" ", "+")
    url = f"https://musicbrainz.org/ws/2/artist/?query=artist:{query}&fmt=json&limit=1"
    
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        if data['artists']:
            a = data['artists'][0]
            datos_artistas.append({
                'artist_name': artist,
                'artist_id': a.get('id'),
                'country': a.get('country'),
                'begin_date': a.get('life-span', {}).get('begin'),
                'end_date': a.get('life-span', {}).get('end'),
                'type': a.get('type'),
                'gender': a.get('gender'),
                'disambiguation': a.get('disambiguation')
            })
    else:
        print(f" Error con {artist}: {response.status_code}")
    
    time.sleep(1)

# Guardar como CSV
df_api = pd.DataFrame(datos_artistas)
df_api.to_csv("api_data.csv")

print("✅ Datos de MusicBrainz guardados en api_data.csv.")

✅ Datos de MusicBrainz guardados en api_data.csv.
